In [1]:
import pandas as pd
import lightgbm as lgb
import numpy as np
import gc
import itertools
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (10, 6)

from importlib import reload
import date
reload(date)
from date import *
import model
reload(model)
from model import *
from metric import get_weights, NWRMSLE_log

In [2]:
df = load_data_in_date_range('./data/train_processed.csv', '2017-04-04', '2017-08-15', 110000000)

Reading batch from position 110000000, batch size 10000000...
Filtering 10000000...
Filtered 8659998, mapping...
Mapped, reducing...
Batch done.
Reading batch from position 120000000, batch size 10000000...
Filtering 5497041...
Filtered 5497041, mapping...
Mapped, reducing...
Batch done.
End of dataset is found.


In [3]:
print('Reading additional datasets')
items = pd.read_csv('./data/items_encoded.csv')
stores = pd.read_csv('./data/stores_encoded.csv')

Reading additional datasets


In [4]:
print('Converting data and joining additional data')
df = convert_unit_sales(df)
df = fill_empty_sales(df)
df, cols_categories = extend_dataset(df, items, stores)
del df['id']
del df['weight']
gc.collect()

Converting data and joining additional data


219

In [5]:
print('Adding lagged features')
df_prev = df[['item_nbr', 'store_nbr', 'date', 'unit_sales']]
df, cols_lagged = fill_lagged(df, df_prev, 12, 18, True)
del df_prev
gc.collect()
df.head()

Adding lagged features


/home/ubuntu/.local/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/ubuntu/grocery/model.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_prev[colname] = df_prev['unit_sales']


Processing lagged 12...
Lagged 12 is done.
Processing lagged 13...
Lagged 13 is done.
Processing lagged 14...
Lagged 14 is done.
Processing lagged 15...
Lagged 15 is done.
Processing lagged 16...
Lagged 16 is done.
Processing lagged 17...
Lagged 17 is done.
Processing lagged 18...
Lagged 18 is done.


,date,store_nbr,item_nbr,unit_sales,onpromotion,weekday,weekend,salary,family,class,...,state,type,cluster,unit_sales(t-12),unit_sales(t-13),unit_sales(t-14),unit_sales(t-15),unit_sales(t-16),unit_sales(t-17),unit_sales(t-18)
0,1635,46,1695836,2.735787,False,5,True,False,30,2004,...,12,0,14,2.995732,3.288285,3.194839,2.603847,2.442347,3.597312,2.956699
1,1636,46,1695836,3.821825,False,6,True,False,30,2004,...,12,0,14,2.545531,2.995732,3.288285,3.194839,2.603847,2.442347,3.597312
2,1637,46,1695836,2.530094,False,0,False,False,30,2004,...,12,0,14,3.076736,2.545531,2.995732,3.288285,3.194839,2.603847,2.442347
3,1638,46,1695836,2.697052,False,1,False,False,30,2004,...,12,0,14,2.978395,3.076736,2.545531,2.995732,3.288285,3.194839,2.603847
4,1639,46,1695836,2.812786,True,2,False,False,30,2004,...,12,0,14,2.652361,2.978395,3.076736,2.545531,2.995732,3.288285,3.194839


In [6]:
cat_features = items_cols + stores_cols + ['store_nbr','item_nbr','onpromotion']
combinations = list(itertools.combinations(cat_features, 1)) + \
[['store_nbr','item_nbr']]

print('Adding mean target encoding')
df, cols_mean = add_mean_encoding(df, combinations)
gc.collect()
!telegram-send "Lagged and mean extraction is ready, starting validation."
df.head()

Adding mean target encoding


,date,store_nbr,item_nbr,unit_sales,onpromotion,weekday,weekend,salary,family,class,...,mean_unit_sales_by_(perishable),mean_unit_sales_by_(family),mean_unit_sales_by_(city),mean_unit_sales_by_(type),mean_unit_sales_by_(cluster),mean_unit_sales_by_(state),mean_unit_sales_by_(store_nbr),mean_unit_sales_by_(item_nbr),mean_unit_sales_by_(onpromotion),mean_unit_sales_by_(store_nbr+item_nbr)
0,1576,46,1695836,3.983762,True,2,False,False,30,2004,...,0.954195,1.029249,1.098829,1.236079,1.307416,1.090374,1.352238,1.758596,1.969504,2.359225
1,1577,46,1695836,2.771612,False,3,False,False,30,2004,...,0.954195,1.029249,1.098829,1.236079,1.307416,1.090374,1.352238,1.758596,0.749576,2.359225
2,1578,46,1695836,3.089621,False,4,False,False,30,2004,...,0.954195,1.029249,1.098829,1.236079,1.307416,1.090374,1.352238,1.758596,0.749576,2.359225
3,1579,46,1695836,3.789149,False,5,True,False,30,2004,...,0.954195,1.029249,1.098829,1.236079,1.307416,1.090374,1.352238,1.758596,0.749576,2.359225
4,1580,46,1695836,4.023117,False,6,True,True,30,2004,...,0.954195,1.029249,1.098829,1.236079,1.307416,1.090374,1.352238,1.758596,0.749576,2.359225


In [7]:
df['date'] = df['date'].astype('int16', copy=False)
df['store_nbr'] = df['store_nbr'].astype('int16', copy=False)
df['item_nbr'] = df['item_nbr'].astype('int32', copy=False)
df['weekday'] = df['weekday'].astype('int16', copy=False)
df['family'] = df['family'].astype('int16', copy=False)
df['class'] = df['class'].astype('int16', copy=False)
df['perishable'] = df['perishable'].astype('bool', copy=False)
df['city'] = df['city'].astype('int16', copy=False)
df['state'] = df['state'].astype('int16', copy=False)
df['type'] = df['type'].astype('int16', copy=False)
df['cluster'] = df['cluster'].astype('int16', copy=False)
gc.collect()

0

In [8]:
df = extract_by_date(df, '2017-05-16', '2017-08-15')
gc.collect()
df.head()

,date,store_nbr,item_nbr,unit_sales,onpromotion,weekday,weekend,salary,family,class,...,mean_unit_sales_by_(perishable),mean_unit_sales_by_(family),mean_unit_sales_by_(city),mean_unit_sales_by_(type),mean_unit_sales_by_(cluster),mean_unit_sales_by_(state),mean_unit_sales_by_(store_nbr),mean_unit_sales_by_(item_nbr),mean_unit_sales_by_(onpromotion),mean_unit_sales_by_(store_nbr+item_nbr)
6,1596,46,1695836,3.227588,False,1,False,True,30,2004,...,0.991647,1.061221,1.116594,1.344079,1.330115,1.108253,1.380306,2.755481,0.770096,3.639252
7,1597,46,1695836,3.950643,True,2,False,False,30,2004,...,0.991647,1.061221,1.116594,1.344079,1.330115,1.108253,1.380306,2.755481,1.963289,3.639252
8,1598,46,1695836,3.029530,False,3,False,False,30,2004,...,0.991647,1.061221,1.116594,1.344079,1.330115,1.108253,1.380306,2.755481,0.770096,3.639252
9,1599,46,1695836,2.682198,False,4,False,False,30,2004,...,0.991647,1.061221,1.116594,1.344079,1.330115,1.108253,1.380306,2.755481,0.770096,3.639252
10,1600,46,1695836,4.010170,False,5,True,False,30,2004,...,0.991647,1.061221,1.116594,1.344079,1.330115,1.108253,1.380306,2.755481,0.770096,3.639252


In [9]:
folds = [('2017-05-16', '2017-06-30', '2017-07-01', '2017-07-15'),
         ('2017-06-01', '2017-07-15', '2017-07-16', '2017-07-31'),
         ('2017-06-16', '2017-07-31', '2017-08-01', '2017-08-15')]

param = {
    'num_leaves':30,
    'objective':'regression_l2',
    'metric':'l2_root',
    'num_threads':4
        }

In [ ]:
results = []
bsts = []

x_cols = cols_categories + ['onpromotion'] + cols_lagged + cols_mean
x_cols = list(set(x_cols) - {'perishable', 'type', 'city', 'family', 'mean_unit_sales_by_(state)', 'mean_unit_sales_by_(city)'})

cat_cols = cols_categories + ['onpromotion']
cat_cols = list(set(cat_cols) - {'perishable', 'type', 'city', 'family'})

for train_start, train_end, test_start, test_end in folds:
    print("Extracting fold...")
    train = extract_by_date(df, train_start, train_end)
    test= extract_by_date(df, test_start, test_end)
    
    print("Preparing train...")
    train_X = train[x_cols]
    train_y = train['unit_sales']
    train_weights = get_weights(train['item_nbr'])
    del train
    gc.collect()
    train_dataset = lgb.Dataset(train_X, label=train_y, weight=train_weights)
    del train_X
    del train_y
    del train_weights
    gc.collect()
    
    print("Preparing test...")
    test_X = test[x_cols].fillna(0.0)
    test_y = test['unit_sales']
    test_weights = get_weights(test['item_nbr'])
    del test
    gc.collect()
    test_dataset = lgb.Dataset(test_X, label=test_y, weight=test_weights, reference=train_dataset)
    gc.collect()

    print("="*80)
    bst = lgb.train(param, 
                    train_dataset, 
                    150,
                    valid_sets=[test_dataset], 
                    early_stopping_rounds=10, 
                    verbose_eval=True, 
                    feature_name=x_cols, 
                    categorical_feature=cat_cols)
    
    del test_dataset
    del train_dataset
    gc.collect()
    
    test_y_pred = bst.predict(test_X)
    error = NWRMSLE_log(test_y_pred, test_y, test_weights)
    print('Validation error: {}'.format(error))
    print("="*80)
    
    results.append(error)
    bsts.append(bst)

Extracting fold...
Preparing train...
Preparing test...


/home/ubuntu/.local/lib/python3.5/site-packages/lightgbm/basic.py:1007: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['class', 'cluster', 'onpromotion', 'salary', 'state', 'weekday', 'weekend']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


In [ ]:
!telegram-send "Mean lagged xgb validation finished. Results: $results"

In [ ]:
lgb.plot_importance(bsts[0])

In [13]:
param['task'] = 'prediction'
start = '2017-07-01'
end = '2017-08-15'
num_round = 100
train = extract_by_date(df, start, end)
train_X = train[x_cols]
train_y = train['unit_sales']
train_weights = get_weights(train['item_nbr'])
train_dataset = lgb.Dataset(train_X, label=train_y, weight=train_weights)
bst = lgb.train(param, 
                train_dataset,
                num_round,
                feature_name=x_cols, 
                categorical_feature=cat_cols)

/home/ubuntu/.local/lib/python3.5/site-packages/lightgbm/basic.py:1007: UserWarning: categorical_feature in Dataset is overrided. New categorical_feature is ['city', 'class', 'cluster', 'family', 'onpromotion', 'perishable', 'salary', 'state', 'type', 'weekday', 'weekend']
  warnings.warn('categorical_feature in Dataset is overrided. New categorical_feature is {}'.format(sorted(list(categorical_feature))))


In [14]:
test = pd.read_csv('./data/test_processed.csv', dtype=types)
test, _ = extend_dataset(test, items, stores)
df_prev = extract_by_date(df, '2017-07-25', '2017-08-15')

test, _ = fill_lagged(test, df_prev, 12, 18)
test, _ = fill_mean_encoding(test, df_prev, combinations)
test.perishable = test.perishable.astype('bool')

test_X = test[x_cols]
test['unit_sales'] = bst.predict(test_X)

/home/ubuntu/.local/lib/python3.5/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value
/home/ubuntu/grocery/model.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df_prev[colname] = df_prev['unit_sales']


In [15]:
test.sort_values(by='id', inplace=True)
test.ix[test.unit_sales < 0, 'unit_sales'] = 0
test['unit_sales'] = np.expm1(test['unit_sales'])
test[['id', 'unit_sales']].to_csv('./submissions/lgb_mean_encoded_lagged_0.54.csv.gz', float_format="%.4f", index=False, compression='gzip')
!telegram-send "Submission is done."

/home/ubuntu/.local/lib/python3.5/site-packages/ipykernel_launcher.py:2: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
